In [104]:
import os
import subprocess
from pathlib import Path
import torch
from mmdet3d.evaluation.metrics import nuscenes_metric as nus_metric
from mmdet3d.evaluation.metrics.nuscenes_metric import output_to_nusc_box
import json 
from pyquaternion import Quaternion
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import transform_matrix
import operator
from functools import reduce
from pathlib import Path
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.eval.detection.config import config_factory
from nuscenes.eval.detection.evaluate import NuScenesEval
from classes import cls_attr_dist, class_names

In [105]:
mini_val_tokens = [
    '3e8750f331d7499e9b5123e9eb70f2e2', 
    '3950bd41f74548429c0f7700ff3d8269', 
    'c5f58c19249d4137ae063b0e9ecd8b8e', 
    '700c1a25559b4433be532de3475e58a9', 
    '747aa46b9a4641fe90db05d97db2acea', 
    'f4f86af4da3b49e79497deda5c5f223a', 
    '6832e717621341568c759151b5974512', 
    'c59e60042a8643e899008da2e446acca', 
    'fa65a298c01f44e7a182bbf9e5fe3697', 
    'a98fba72bde9433fb882032d18aedb2e', 
    'b6b0d9f2f2e14a3aaa2c8aedeb1edb69', 
    '796b1988dd254f74bf2fb19ba9c5b8c6', 
    '0d0700a2284e477db876c3ee1d864668', 
    'de7593d76648450e947ba0c203dee1b0', 
    'f753d3f87e5b40af87ff2cbf7c8e7082', 
    'b22fa0b3c34f47b6a360b60f35d5d567', 
    '0a0d6b8c2e884134a3b48df43d54c36a', 
    '8e9c2cba0ee74056aa3746e8391d54a9', 
    'fdc39b23ab4242eda6ec5e1e6574fe33', 
    '9ee4020153674b9e9943d395ff8cfdf3', 
    '5b03af7a953245b5a3b23191ed4da62a', 
    'd0ecf474e0e64950aa265cd44b9c9d75', 
    '456ec36cb4a44ca78f36fbd90c0c34fa', 
    '6bfd42cf0aba4f1a94ec11fa43e2dd92', 
    '0af0feb5b1394b928dd13d648de898f5', 
    '61d9340c5ad8418dafe7a4af1b96e6b9', 
    '9e7683e8586542a1b6032980c45f15ce', 
    'cc57c1ea80fe46a7abddfdb15654c872', 
    '4cf5d6c3f6ab42aab23f67b5a9782d1a', 
    '6d9984d09d52479e837da2fd09e192cc', 
    '38a28a3aaf2647f2a8c0e90e31267bf8', 
    '6402fd1ffaf041d0b9162bd92a7ba0a2', 
    'ce94ef7a0522468e81c0e2b3a2f1e12d', 
    '9fcdc52b791045e99c623c5fc643331f', 
    '1c9a906c40f6417bbe1cea06d6e55501', 
    'a771effa2a2648d78096c3e92b95b129', 
    'b9ea04a6121d4a8bb00199b885aa5ef0', 
    '87e772078a494d42bd34cd16172808bc', 
    'f40544fd4f5d42abbcfa948eeaf86850', 
    '281b92269fd648d4b52d06ac06ca6d65', 
    'b5989651183643369174912bc5641d3b', 
    '0bb62a68055249e381b039bf54b0ccf8', 
    '07fad91090c746ccaa1b2bdb55329e20', 
    'ae5004bf4ebb4db0a84cb3c27bd398d1', 
    'ac8b4d49731d43289579f014cb7c97ef', 
    'c1eed31234b94e9f8e22fbf3428b0ac2', 
    'edba7d0cf1c64d2a9c8a5ebddab507a9', 
    'a5afebb0aa5e4d7c95665788ce51ec58', 
    '5b7cb170eee6468aa1fdbd3abcf63c5a', 
    'd8251bbc2105497ab8ec80827d4429aa', 
    '048a45dd2cf54aa5808d8ccc85731d44', 
    '858a1ece22cf45d9bc71e42336604b78', 
    '372725a4b00e49c78d6d0b1c4a38b6e0', 
    '61a7bd24f88a46c2963280d8b13ac675', 
    '609d5177362340458a3bfd4949cd1e64', 
    'b1303058fa624645b753d7283d70de45', 
    'a19a80c905674faab7203a3a4e0f5246', 
    'b6c420c3a5bd4a219b1cb82ee5ea0aa7', 
    '8092909473464f80b9f791a4d31ddcb8', 
    'e174cb43655f49dab7ffa27b973670e3', 
    '6ff9723a60bf4e14b328b3b19f04dc32', 
    '06be0e3b665c44fa8d17d9f4770bdf9c', 
    '9150678870764c1b87a649a25939c61b', 
    '8573a885a7cb41d185c05029eeb9a54e', 
    '55c258972acb4300a3a6077a531ab050', 
    '44237858a539457da65822bfcf58c414', 
    '8cd36e9531fb4eba8e6ac1d666c4641c', 
    '7bebe3c9be714f02837f8617c56df122', 
    '4b894442c95141f9affd731d9da7b43c', 
    'e00dc15130dc44e796687baadd076ae4', 
    'e63b83b436a0479db7362338cdfab118', 
    'c567f9b8e9f34817acdc9c49d791c557', 
    '8e352d4a6c6f40c8ad4f10a3d2c3f158', 
    '841dd6709e9b4d7c9d6bf888f1fe6d7e', 
    'e54b784a608644a2804fcf800c7499f7', 
    'a1289b27ca1d41deb6fc982be9a3d03c', 
    'e6e877f31dd447199b56cae07f86daad', 
    'cb4e6195faad467094fbd4d0a9e960e9', 
    '7f594234e8034228b1a7d727f1981e09', 
    '9cdbf5ff7f294549aea0a4307e5d104a', 
    'b4ff30109dd14c89b24789dc5713cf8c']

In [106]:
backend_args = None
home_dir = str(Path.home())
nusc = NuScenes(version='v1.0-mini', dataroot = f"{home_dir}/software/mmdetection3d/data/nuscenes")
dataroot = f"{home_dir}/software/mmdetection3d/data/nuscenes/"
out_dir = f"{home_dir}/nuscenes_dataset/inference_results_mini"
preds_dir = os.path.join(out_dir, "preds")
ann_file=dataroot + 'nuscenes_infos_val.pkl'
metric='bbox'

pcd_path = f"{home_dir}/software/mmdetection3d/data/nuscenes/samples/LIDAR_TOP/"
mmdet_path = f"{home_dir}/software/mmdetection3d"
pcd_list = os.listdir(pcd_path)

# Config and model:
configs_path = "configs/pointpillars/pointpillars_hv_fpn_sbn-all_8xb4-2x_nus-3d.py"
checkpoint_path = "checkpoints/hv_pointpillars_fpn_sbn-all_4x8_2x_nus-3d_20210826_104936-fca299c1.pth"

# Instantiate evaluator:
evaluator = nus_metric.NuScenesMetric(dataroot, ann_file)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.416 seconds.
Reverse indexing ...
Done reverse indexing in 0.2 seconds.


In [107]:
# Modified PCDet functions:
# Box conversion
def boxes_lidar_to_nusenes(det_info):
    boxes3d = det_info['bboxes_3d']
    scores = det_info['scores_3d']
    labels = det_info['labels_3d']

    box_list = []
    for k in range(boxes3d.shape[0]):
        quat = Quaternion(axis=[0, 0, 1], radians=boxes3d[k, 6])
        velocity = (*boxes3d[k, 7:9], 0.0) if boxes3d.shape[1] == 9 else (0.0, 0.0, 0.0)
        box = Box(
            boxes3d[k, :3],
            boxes3d[k, [4, 3, 5]],  # wlh
            quat, label=labels[k], score=scores[k], velocity=velocity,
        )
        box_list.append(box)
    return box_list

def lidar_nusc_box_to_global(nusc, boxes, sample_token):
    s_record = nusc.get('sample', sample_token)
    sample_data_token = s_record['data']['LIDAR_TOP']

    sd_record = nusc.get('sample_data', sample_data_token)
    cs_record = nusc.get('calibrated_sensor', sd_record['calibrated_sensor_token'])
    sensor_record = nusc.get('sensor', cs_record['sensor_token'])
    pose_record = nusc.get('ego_pose', sd_record['ego_pose_token'])

    data_path = nusc.get_sample_data_path(sample_data_token)
    box_list = []
    for box in boxes:
        # Move box to ego vehicle coord system
        box.rotate(Quaternion(cs_record['rotation']))
        box.translate(np.array(cs_record['translation']))
        # Move box to global coord system
        box.rotate(Quaternion(pose_record['rotation']))
        box.translate(np.array(pose_record['translation']))
        box_list.append(box)
    return box_list

def transform_det_annos_to_nusc_annos(det_annos, nusc):
    nusc_annos = {
        'results': {},
        'meta': None,
    }

    for det in det_annos:
        annos = []
        try:
            box_list = boxes_lidar_to_nusenes(det)
            box_list = lidar_nusc_box_to_global(
                nusc=nusc, boxes=box_list, sample_token=det['metadata']['token']
            )
        except:
            print("TypeError in transform_det_annos_to_nusc_annos: string indices must be integers")
        for k, box in enumerate(box_list):
            name = det['name'][k]
            if np.sqrt(box.velocity[0] ** 2 + box.velocity[1] ** 2) > 0.2:
                if name in ['car', 'construction_vehicle', 'bus', 'truck', 'trailer']:
                    attr = 'vehicle.moving'
                elif name in ['bicycle', 'motorcycle']:
                    attr = 'cycle.with_rider'
                else:
                    attr = None
            else:
                if name in ['pedestrian']:
                    attr = 'pedestrian.standing'
                elif name in ['bus']:
                    attr = 'vehicle.stopped'
                else:
                    attr = None
            attr = attr if attr is not None else max(
                cls_attr_dist[name].items(), key=operator.itemgetter(1))[0]
            nusc_anno = {
                'sample_token': det['metadata']['token'],
                'translation': box.center.tolist(),
                'size': box.wlh.tolist(),
                'rotation': box.orientation.elements.tolist(),
                'velocity': box.velocity[:2].tolist(),
                'detection_name': name,
                'detection_score': box.score,
                'attribute_name': attr
            }
            if det['scores_3d'][k] >= 0.6:
                annos.append(nusc_anno)

        nusc_annos['results'].update({det["metadata"]["token"]: annos})

    return nusc_annos


In [108]:
def construct_token_dict():
    """
    Constructs a dictionary mapping filenames to lidar and sample tokens
    """
    token_dict = dict()
    for scene in nusc.scene:
        sample_token = scene['first_sample_token']
        sample = nusc.get('sample', sample_token)
        lidar_data = nusc.get('sample_data', sample['data']["LIDAR_TOP"])
        while sample['next'] != "":
            filename = lidar_data['filename']
            file_str = filename[filename.rfind("/")+1:].replace("bin", "json")
            token_dict[file_str] = {"lidar_token": lidar_data['token'], "sample_token": lidar_data['sample_token']}
            sample = nusc.get("sample", sample['next'])
            lidar_data = nusc.get('sample_data', sample['data']["LIDAR_TOP"])
        
        filename = lidar_data['filename']
        file_str = filename[filename.rfind("/")+1:].replace("bin", "json")
        token_dict[file_str] = {"lidar_token": lidar_data['token'], "sample_token": lidar_data['sample_token']}

    with open("token_dict_mini.json", 'w') as f:
        json.dump(token_dict, f)
    f.close()

In [109]:
def get_sample_token(fn):
    with open("token_dict_mini.json", 'r') as f:
        token_dict = json.load(f)
    sample_token = token_dict[fn]['sample_token']
    f.close()
    return sample_token

# Read json file:
def read_preds_file(fn):
    full_fn = os.path.join(preds_dir, fn)
    with open(full_fn, 'r') as f:
        result = json.load(f)
        result['bboxes_3d'] = torch.Tensor(result['bboxes_3d']).numpy()
        result['scores_3d'] = torch.Tensor(result['scores_3d']).numpy()
        class_labels = [class_names[k] for k in result['labels_3d']]
        result['labels_3d'] = torch.Tensor(result['labels_3d']).numpy()
        sample_token = dict()
        sample_token['token'] = get_sample_token(fn)
        result.update({'metadata':sample_token})
        result.update({'name':class_labels})
        
    f.close()
    return result

def read_results():
    """
    Reads the results from prediction files.

    Returns:
        list: A list of dictionaries containing the processed prediction data.
    """
    preds_fn = os.listdir(preds_dir)
    results = []
    count = 1
    for fn in preds_fn:
        if count%1000 == 0:
            print("Read results count: ", str(count))
        results.append(read_preds_file(fn))
        count += 1
    return results

def custom_result(fn):
    results = [read_preds_file(fn)]
    return results


In [110]:
def save_nusc_results(det_annos, **kwargs):
    nusc_annos = transform_det_annos_to_nusc_annos(det_annos, nusc)
    nusc_annos['meta'] = {
        'use_camera': False,
        'use_lidar': True,
        'use_radar': False,
        'use_map': False,
        'use_external': False,
    }

    output_path = Path(kwargs['output_path'])
    output_path.mkdir(exist_ok=True, parents=True)
    res_path = str(output_path / 'results_nusc.json')
    with open(res_path, 'w') as f:
        json.dump(nusc_annos, f)
    
    print('The predictions of NuScenes have been saved to {res_path}')
    return output_path, res_path

def get_metrics(output_path, res_path):
    eval_set_map = {
        'v1.0-mini': 'mini_val',
        'v1.0-trainval': 'val',
        'v1.0-test': 'test'
    }
    try:
        eval_version = 'detection_cvpr_2019'
        eval_config = config_factory(eval_version)
    except:
        eval_version = 'cvpr_2019'
        eval_config = config_factory(eval_version)

    nusc_eval = NuScenesEval(
        nusc,
        config=eval_config,
        result_path=res_path,
        eval_set=eval_set_map['v1.0-mini'],
        output_dir=str(output_path),
        verbose=True,
    )
    metrics_summary = nusc_eval.main(plot_examples=0, render_curves=False)

    with open(output_path / 'metrics_summary.json', 'r') as f:
        metrics = json.load(f)
    return metrics, metrics_summary, nusc_eval

In [111]:
def filter_results(results):
    return [
        result
        for result in results
        if result['metadata']['token'] in mini_val_tokens
    ]

In [112]:
construct_token_dict()
results = read_results()
results = filter_results(results)
nusc_results = transform_det_annos_to_nusc_annos(results, nusc)

In [113]:
print(len(nusc_results))
output_path, res_path = save_nusc_results(results, output_path="/home/ranai/nuscenes_dataset/3D_Detection")
metrics, metrics_summary = get_metrics(output_path, res_path)

2
The predictions of NuScenes have been saved to {res_path}
Initializing nuScenes detection evaluation
Loaded results from /home/ranai/nuscenes_dataset/3D_Detection/results_nusc.json. Found detections for 81 samples.
Loading annotations for mini_val split from nuScenes version: v1.0-mini


100%|██████████| 81/81 [00:00<00:00, 269.82it/s]


Loaded ground truth annotations for 81 samples.
Filtering predictions
=> Original number of boxes: 905
=> After distance based filtering: 905
=> After LIDAR and RADAR points based filtering: 905
=> After bike rack filtering: 905
Filtering ground truth annotations
=> Original number of boxes: 4441
=> After distance based filtering: 3785
=> After LIDAR and RADAR points based filtering: 3393
=> After bike rack filtering: 3393
Accumulating metric data...
Calculating metrics...
Saving metrics to: /home/ranai/nuscenes_dataset/3D_Detection
mAP: 0.0755
mATE: 0.8586
mASE: 0.8272
mAOE: 0.8025
mAVE: 1.2312
mAAE: 0.8557
NDS: 0.1034
Eval time: 1.2s

Per-class results:
Object Class        	AP    	ATE   	ASE   	AOE   	AVE   	AAE   
car                 	0.361 	0.175 	0.151 	0.178 	0.364 	0.072 
truck               	0.000 	1.000 	1.000 	1.000 	1.000 	1.000 
bus                 	0.394 	0.411 	0.121 	0.045 	3.486 	0.773 
trailer             	0.000 	1.000 	1.000 	1.000 	1.000 	1.000 
construction_vehicle	

ValueError: too many values to unpack (expected 2)